# EVP-KLEE: Enhanced Value Profiling for KLEE - Google Colab Demo

This notebook demonstrates the EVP-KLEE automated symbolic execution pipeline in Google Colab.

## Overview
EVP-KLEE is a three-phase pipeline that enhances KLEE with value profiling capabilities:
1. **Phase 1**: Extract bitcode and instrument programs
2. **Phase 2**: Run tests and collect value profiles  
3. **Phase 3**: Run KLEE with and without EVP enhancements

## Setup Instructions
1. Mount your Google Drive
2. Upload the EVP-KLEE project files
3. Install dependencies
4. Run the automated demo


## 1. Environment Setup


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive mounted successfully!")
print("Please upload your EVP-KLEE project to Google Drive first.")
print("Expected path: /content/drive/MyDrive/EVP-KLEE/")


In [ ]:
# Install system dependencies
!apt update
!apt install -y build-essential cmake git wget curl vim nano htop time \
    python3 python3-pip python3-venv python3-dev pkg-config \
    libncurses5-dev libncursesw5-dev zlib1g-dev libedit-dev \
    libxml2-dev libzstd-dev libsqlite3-dev libssl-dev libffi-dev \
    libbz2-dev libreadline-dev libgdbm-dev liblzma-dev libtinfo-dev \
    libc6-dev libc++-dev libc++abi-dev

print("System dependencies installed!")


In [ ]:
# Install LLVM 10 and Clang 10
!apt install -y llvm-10 llvm-10-dev llvm-10-tools clang-10 libclang-10-dev

# Create symlinks for easier access
!ln -s /usr/bin/llvm-config-10 /usr/bin/llvm-config
!ln -s /usr/bin/clang-10 /usr/bin/clang
!ln -s /usr/bin/clang++-10 /usr/bin/clang++
!ln -s /usr/lib/llvm-10 /usr/lib/llvm

print("LLVM 10 and Clang 10 installed!")


In [ ]:
# Install STP Solver
!cd /tmp && git clone https://github.com/stp/stp.git
!cd /tmp/stp && git checkout 2.3.3
!cd /tmp/stp && mkdir build && cd build
!cd /tmp/stp/build && cmake -DCMAKE_BUILD_TYPE=Release -DBUILD_SHARED_LIBS=ON ..
!cd /tmp/stp/build && make -j$(nproc)
!cd /tmp/stp/build && make install
!ldconfig
!rm -rf /tmp/stp

print("STP Solver installed!")


In [ ]:
# Install Z3 Solver
!cd /tmp && wget https://github.com/Z3Prover/z3/releases/download/z3-4.8.12/z3-4.8.12-x64-ubuntu-18.04.zip
!cd /tmp && unzip z3-4.8.12-x64-ubuntu-18.04.zip
!cd /tmp && cp z3-4.8.12-x64-ubuntu-18.04/bin/z3 /usr/local/bin/
!cd /tmp && cp z3-4.8.12-x64-ubuntu-18.04/bin/libz3.so /usr/local/lib/
!ldconfig
!rm -rf /tmp/z3-4.8.12-x64-ubuntu-18.04*

print("Z3 Solver installed!")


In [ ]:
# Install Python dependencies
%pip install --upgrade pip
%pip install wllvm
%pip install matplotlib seaborn pandas numpy scipy
%pip install jupyter ipywidgets

print("Python dependencies installed!")


## 2. Project Setup


In [ ]:
import os
import sys
from pathlib import Path
import shutil

# Set up project directory
PROJECT_ROOT = "/content/EVP-KLEE"
DRIVE_PATH = "/content/drive/MyDrive/EVP-KLEE"

# Check if project exists in Drive
if os.path.exists(DRIVE_PATH):
    print(f"Found EVP-KLEE project in Drive: {DRIVE_PATH}")
    # Copy project to local workspace
    if os.path.exists(PROJECT_ROOT):
        shutil.rmtree(PROJECT_ROOT)
    shutil.copytree(DRIVE_PATH, PROJECT_ROOT)
    print(f"Project copied to: {PROJECT_ROOT}")
else:
    print(f"EVP-KLEE project not found in Drive at: {DRIVE_PATH}")
    print("Please upload your project to Google Drive first.")
    print("You can use the upload button in the file browser or run:")
    print("!git clone https://github.com/roxaw/evp-klee-artifact.git /content/EVP-KLEE")

# Change to project directory
os.chdir(PROJECT_ROOT)
print(f"Current directory: {os.getcwd()}")

# List project contents
print("\nProject contents:")
!ls -la


In [ ]:
# Build KLEE (this may take 10-15 minutes)
print("Building KLEE... This may take 10-15 minutes.")
!bash scripts/build_klee.sh

print("KLEE build completed!")


## 3. EVP-KLEE Automated Demo


In [ ]:
# Run the automated demo
import subprocess
import json
from datetime import datetime

print("Starting EVP-KLEE Automated Demo...")
print(f"Timestamp: {datetime.now()}")
print("=" * 60)

# Change to automated_demo directory
os.chdir("automated_demo")

# Run the pipeline
result = subprocess.run(["python3", "evp_pipeline.py", "coreutils"], 
                       capture_output=True, text=True, timeout=3600)

print("STDOUT:")
print(result.stdout)

if result.stderr:
    print("\nSTDERR:")
    print(result.stderr)

print(f"\nReturn code: {result.returncode}")


## 4. Results Analysis


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Find results directory
results_dir = Path("../benchmarks/evp_artifacts")
if results_dir.exists():
    print(f"Results directory found: {results_dir}")
    print("\nResults structure:")
    !find {results_dir} -name "*.json" -o -name "*.txt" | head -20
else:
    print("Results directory not found. Demo may not have completed successfully.")

# Look for KLEE results
klee_results = list(results_dir.glob("**/klee_results_*.json"))
if klee_results:
    print(f"\nFound {len(klee_results)} KLEE result files:")
    for result_file in klee_results:
        print(f"  - {result_file}")
else:
    print("\nNo KLEE results found.")


In [ ]:
# Visualize results if available
if klee_results:
    # Load and analyze results
    results_data = []
    
    for result_file in klee_results:
        with open(result_file, 'r') as f:
            data = json.load(f)
            results_data.append(data)
    
    # Create comparison plots
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('EVP-KLEE Performance Comparison', fontsize=16)
    
    programs = [r['program'] for r in results_data]
    vanilla_tests = [r['vanilla']['ktest_count'] for r in results_data]
    evp_tests = [r['evp']['ktest_count'] for r in results_data]
    
    # Test case count comparison
    axes[0,0].bar(programs, vanilla_tests, alpha=0.7, label='Vanilla KLEE')
    axes[0,0].bar(programs, evp_tests, alpha=0.7, label='EVP KLEE')
    axes[0,0].set_title('Test Cases Generated')
    axes[0,0].set_ylabel('Number of Test Cases')
    axes[0,0].legend()
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # Performance improvement
    improvements = [(v-e)/v*100 if v > 0 else 0 for v, e in zip(vanilla_tests, evp_tests)]
    axes[0,1].bar(programs, improvements, color='green', alpha=0.7)
    axes[0,1].set_title('Performance Improvement (%)')
    axes[0,1].set_ylabel('Improvement %')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # Success rate
    vanilla_success = [1 if r['vanilla']['success'] else 0 for r in results_data]
    evp_success = [1 if r['evp']['success'] else 0 for r in results_data]
    
    x = range(len(programs))
    width = 0.35
    axes[1,0].bar([i - width/2 for i in x], vanilla_success, width, label='Vanilla KLEE', alpha=0.7)
    axes[1,0].bar([i + width/2 for i in x], evp_success, width, label='EVP KLEE', alpha=0.7)
    axes[1,0].set_title('Success Rate')
    axes[1,0].set_ylabel('Success (1) / Failure (0)')
    axes[1,0].set_xticks(x)
    axes[1,0].set_xticklabels(programs, rotation=45)
    axes[1,0].legend()
    
    # Summary statistics
    axes[1,1].text(0.1, 0.8, f'Total Programs: {len(programs)}', transform=axes[1,1].transAxes, fontsize=12)
    axes[1,1].text(0.1, 0.7, f'Vanilla Success: {sum(vanilla_success)}/{len(programs)}', transform=axes[1,1].transAxes, fontsize=12)
    axes[1,1].text(0.1, 0.6, f'EVP Success: {sum(evp_success)}/{len(programs)}', transform=axes[1,1].transAxes, fontsize=12)
    axes[1,1].text(0.1, 0.5, f'Avg Improvement: {sum(improvements)/len(improvements):.1f}%', transform=axes[1,1].transAxes, fontsize=12)
    axes[1,1].set_title('Summary Statistics')
    axes[1,1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed results
    print("\nDetailed Results:")
    print("=" * 80)
    for data in results_data:
        print(f"\nProgram: {data['program']}")
        print(f"  Vanilla KLEE: {'SUCCESS' if data['vanilla']['success'] else 'FAILED'}")
        print(f"    - Test cases: {data['vanilla']['ktest_count']}")
        print(f"  EVP KLEE: {'SUCCESS' if data['evp']['success'] else 'FAILED'}")
        print(f"    - Test cases: {data['evp']['ktest_count']}")
        if data['vanilla']['success'] and data['evp']['success']:
            improvement = (data['vanilla']['ktest_count'] - data['evp']['ktest_count']) / data['vanilla']['ktest_count'] * 100
            print(f"    - Improvement: {improvement:.2f}%")
else:
    print("No results available for visualization.")


## 5. Save Results to Google Drive


In [ ]:
# Save results back to Google Drive
import shutil
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_backup = f"/content/drive/MyDrive/EVP-KLEE-Results-{timestamp}"

if os.path.exists("../benchmarks/evp_artifacts"):
    shutil.copytree("../benchmarks/evp_artifacts", f"{results_backup}/evp_artifacts")
    print(f"Results saved to: {results_backup}")
    
    # Also save the notebook output
    shutil.copy("evp_colab_demo.ipynb", f"{results_backup}/")
    print("Notebook saved to Drive as well.")
else:
    print("No results to save.")

print("\nDemo completed! Check your Google Drive for results.")


## 6. Cursor IDE Integration

To integrate this with Cursor IDE:

1. **Download the notebook**: Save this notebook to your local machine
2. **Open in Cursor**: Open the notebook file in Cursor IDE
3. **Install Jupyter extension**: Install the Jupyter extension in Cursor
4. **Connect to Colab**: Use the local runtime connection method
5. **Sync with Drive**: Your results will be automatically synced to Google Drive

### Alternative: Direct Cursor Integration

You can also run this directly in Cursor by:
1. Installing the required dependencies locally
2. Cloning the EVP-KLEE repository
3. Running the automated demo directly

This approach gives you the full power of Cursor's AI assistance while working with your EVP-KLEE project.
